# Prepare the Catalogs

Important functions that prepare the downloaded catalogs for the cross-matching, soms and SED fitting

In [1]:
# Libraries
from astropy.table import Table

In [ ]:
# Insert the path of the downloaded catalogs
emu_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'
des_catalog_path = '/data/mfonseca/survey_data/DES_data/DES_in_EMU_0102-32_square.fits'
viking_catalog_path = '/data/mfonseca/survey_data/VIKING_data/VIKING_in_EMU_0102-32.fits'
catwise_catalog_path = '/data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits'

## Convert VEGA magnitude to AB magnitude (VIKING, CATWISE)

In [ ]:
def fix_ab_magnitude(file_path, ab_offset, mag_columns):
    '''
    Converts the vega magnitudes to ab magnitudes

    Args:
        file_path (str): Path to the FITS file.
        ab_offset (list): List of offsets for each magnitude column.
        mag_columns (list): List of magnitude column names to be corrected, same order as ab_offset.
    Returns:
        None: The function modifies the FITS file in place.
    
    '''

    catalog_table = Table.read(file_path)
    catalog_df = catalog_table.to_pandas()

    for col in mag_columns:
        # Apply the AB magnitude correction
        index = mag_columns.index(col)
        catalog_df[col] = catalog_df[col] + ab_offset[index]


    # Save the updated DataFrame back to the original FITS file
    updated_catalog_table = Table.from_pandas(catalog_df)
    updated_catalog_table.write(file_path, overwrite=True)

In [ ]:
catwise_magnitude_columns = ['w1mpro', 'w2mpro']
viking_magnitude_columns = ['zPetroMag', 'yPetroMag', 'jPetroMag', 'hPetroMag', 'ksPetroMag']

# https://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html
catwise_ab_offset = [2.699, 3.339]
# http://casu.ast.cam.ac.uk/surveys-projects/vista/technical/filter-set
viking_ab_offset = [0.502, 0.600, 0.916, 1.366, 1.827]

In [ ]:
fix_ab_magnitude(catwise_catalog_path, catwise_ab_offset, catwise_magnitude_columns)
fix_ab_magnitude(viking_catalog_path, viking_ab_offset, viking_magnitude_columns)

## Calculate Colors (VIKING, CATWISE)

In [ ]:
# For the VIKING catalog
catalog_table = Table.read(viking_catalog_path)
viking_catalog_df = catalog_table.to_pandas()

viking_catalog_df['z_y'] = viking_catalog_df['zPetroMag'] - viking_catalog_df['yPetroMag']
viking_catalog_df['y_j'] = viking_catalog_df['yPetroMag'] - viking_catalog_df['jPetroMag']
viking_catalog_df['j_h'] = viking_catalog_df['jPetroMag'] - viking_catalog_df['hPetroMag']
viking_catalog_df['h_ks'] = viking_catalog_df['hPetroMag'] - viking_catalog_df['ksPetroMag']

updated_catalog_table = Table.from_pandas(viking_catalog_df)
updated_catalog_table.write(viking_catalog_path, overwrite=True)

In [ ]:
# For the CATWISE catalog
catwise_catalog_table = Table.read(catwise_catalog_path)
catwise_catalog_df = catwise_catalog_table.to_pandas()

catwise_catalog_df['w1_w2'] = catwise_catalog_df['w1mpro'] - catwise_catalog_df['w2mpro']

updated_catalog_table = Table.from_pandas(catwise_catalog_df)
updated_catalog_table.write(catwise_catalog_path, overwrite=True)